In [21]:
import pandas as pd
import numpy as np
import re, os, nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
path = r"C:\Users\crisp\Downloads\IMDB Dataset.csv\IMDB Dataset.csv"
movies = pd.read_csv(path)
movies.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [22]:
def label_encoder(sentiment):
    return 1 if sentiment == "positive" else 0
def clean_review(review):
    d = re.sub(r'<[^>]+>', ' ', review)
    a = re.sub(r'[^a-zA-Z]',' ', d)
    b = re.sub(r'\s+', ' ', a)
    c = re.sub(r'\s[^AaIi]{1}\s', ' ', b).lower()
    return c

In [23]:
sentiments = np.array(list(map(label_encoder, movies.sentiment.values)))
reviews = np.array(list(map(clean_review, movies.review.values)))

In [24]:
sentiments[:5]

array([1, 1, 1, 0, 1])

In [25]:
reviews[:5]

array(['one of the other reviewers has mentioned that after watching just oz episode you ll be hooked they are right as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to many aryans muslims gangstas latinos christians italians irish and more so scuffles death stares dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wouldn dare forget pre

In [4]:
X = []
y = []

count_pos = count_neg = 0
for sent, rev in zip(sentiments, reviews):
    if sent == 1 and count_pos != 10000:
        y.append(sent)
        X.append(rev)
        count_pos +=1
    
    if sent == 0 and count_neg != 10000:
        y.append(sent)
        X.append(rev)   
        count_neg +=1
        
    if count_neg == 10000 and count_pos == 10000:
        break
        
    

In [5]:
len(X), len(y)

(20000, 20000)

In [6]:
reviews.shape

(50000,)

In [7]:

""""
Splitting the data into training and testing
"""
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=.15, random_state=33)
print(X_train.shape, X_test.shape, y_train.shape)

(17000,) (3000,) (17000,)


> Trainning the Classifier

In [8]:
pipe = Pipeline([
    ('vect', CountVectorizer(
        lowercase=True,
        stop_words="english",
        ngram_range=(1, 2),
        binary=True
    )),
    ('clf', MultinomialNB())
])

pipe.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(binary=True, ngram_range=(1, 2),
                                 stop_words='english')),
                ('clf', MultinomialNB())])

In [9]:
print(pipe.score(X_test, y_test))

0.8646666666666667


In [10]:
import pickle

In [12]:
with open('./models/sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(pipe, f)

print("MODEL SAVED")

MODEL SAVED


In [13]:
with open('models/sentiment_classifier.pkl', 'rb') as f:
    model = pickle.load(f)

In [15]:
X_test[0]

'i watched gomeda on movie theater at my city my friend took away me and i was really curious what would be it looked like well i must say this movie was not a horror may be we can say that is fantsastic experimentation ok here i go anyway but there was a lot of shooting acting dramatic theatrical and storytelling problems i can understand because of director is very young and gomeda is his first feature film ok directing of this film was not pretty bad i see unfortunately due to the restraints placed on the film by its extremely low budget the visuals are often as murky as the storyline and there is no powerful gothic scenes as a horror movie it really fails no scares at all and it is quite muddled and boring some people say gomeda is an art movie but i could not see a laughable terrible and breoken off art movie like that so how can we say it is an art movie just funny '

In [19]:
model.predict(X_test[:5]), y_test[:5]

(array([0, 1, 1, 1, 1]), array([0, 1, 1, 1, 1]))

In [ ]:
pipe2 = Pipeline([
    ('vect', CountVectorizer(
        lowercase=True,
        stop_words="english",
        ngram_range=(1, 2),
        binary=True
    )),
    ('clf', SVC())
])

pipe2.fit(X_train, y_train)

In [ ]:
print(pipe2.score(X_test, y_test))